In [1]:
from llm_generator import *
from create_combinations import *

In [2]:
# Path variables
MODEL_PATH = Path('../results/QuatE/trained_model.pkl')
TRIPLES_PATH = Path('../data/triples_new_without_ct_ss.csv')
RECIPE_PATH_CSV = Path("../data/dataFullLargerRegionAndCountryWithServingsBin.csv")
JSON_COMBINATIONS_PATH = 'evaluation/combinations'

In [3]:
# Create random combinations of user criteria
combos_without_ingredients, combos_with_ingredients = \
    create_combinations(RECIPE_PATH_CSV, JSON_COMBINATIONS_PATH, 3)

print(f"# Processing combo without ingredients: {len(combos_without_ingredients)}")
print(f"# Processing combo with ingredients: {len(combos_with_ingredients)}")

# Processing combo without ingredients: 9
# Processing combo with ingredients: 9


In [4]:
def get_results(combos):    
    results = []

    for i, uc in enumerate(combos):
        
        print(f"Run #{i+1} — Processing combo: {uc}")
        
        # Get recipe generation by LLM
        prompt = get_prompt(uc, MODEL_PATH, TRIPLES_PATH, RECIPE_PATH_CSV)
        messages = create_messages(prompt)
        response = get_response("deepseek/deepseek-chat-v3-0324:free", messages)
        raw_text = response.strip()
        json_text = get_json(raw_text)

        # LLM-as-a-judge: evaluate the recipe generation by another LLM
        system_prompt, user_prompt = get_prompt_evaluation(json_text)
        messages_eval = create_messages(system_prompt, user_prompt)
        response_eval = get_response("deepseek/deepseek-chat-v3-0324:free", messages_eval)
        raw_text_eval = response_eval.strip()
        json_text_eval = get_json_evaluation(raw_text_eval)

        # Save results
        scores = json.loads(json_text_eval)
        results.append(scores)

        break

    return pd.DataFrame(results)

In [5]:
df_without_ingredients = get_results(combos_without_ingredients)
df_without_ingredients.to_csv(JSON_COMBINATIONS_PATH + '_without_ingredients_results.csv')
df_with_ingredients = get_results(combos_with_ingredients)
df_with_ingredients.to_csv(JSON_COMBINATIONS_PATH + '_with_ingredients_results.csv')

Run #1 — Processing combo: {'meal_type': ['snack']}
Run #1 — Processing combo: {'ingredients': ['Salad cucumber']}
